### Youtube Crawling ver.3 
- 이번에는 url 수집 (검색어 입력, 클롤링 글 개수 입력) 후 해당 채널의 제목, 조회수, 날짜, 좋아요, 싫어요, 댓글 등을 수집 

### step 0. 준비
- 필요한 패키지 임포트 

In [1]:
import sys  # 시스템 - 인터프리터를 제어할 수 있는 모듈
import os   # 시스템 - 운영체제를 제어할 수 있는 모듈 

import pandas as pd  # 판다스 임포트
import numpy as np   # 넘파이 임포트 
import openpyxl      # 파이썬에서 엑셀(xls, xlsx, xlsm)을 다룰 수 있는 패키지

from selenium import webdriver    # 웹 프라우저 자동화
from selenium.webdriver.chrome.options import Options  # 크롬 드라이버 옵션 모듈
import time                       # 시간 지연 
import math                       # 수학적인 문제(수학공식 계산)를 해결해주는 모듈 
from selenium.webdriver.common.keys import Keys  # key(키보드의 키를 컴퓨터에게 전달하기 위한) 모듈 
from tqdm import tqdm             # 진행상황 표시 

### step 1. url 수집 

In [3]:
# 원하는 키워드와 크롤링 개수 입력 변수 만들기
keyword = input("검색어 입력: ")                 # 키워드 입력
articles_number = int(input("크롤링 글 개수: ")) # 클롤링 개수 입력 

# 크롬 웹브라우저 실행 
browser = webdriver.Chrome()  # 크롬 드라이버 실행
browser.get("https://www.youtube.com/results?search_query={}".format(keyword))  # 접속하고자 하는 URL 띄우기
time.sleep(2)                 # 2초간 프로세스 중지 

검색어 입력: 무지 반팔티
크롤링 글 개수: 10


In [3]:
# 스크롤 다운 함수
def scroll_down(browser):
    browser.execute_script("window.scrollTo(0, 99999)")
    time.sleep(2)

In [4]:
# 영상 목록 스크롤 다운 실행 
# 소수점을 올려서 정수화 시킴 - math ceil
for i in range(math.ceil((articles_number-26)/20)):  
    scroll_down(browser)

In [5]:
crawling_list = []
url_list = []

# url 수집 
crawling_list = browser.find_elements_by_tag_name('h3 > a')

for crawling in crawling_list:
    url = crawling.get_attribute('href')
    url_list.append(url)
    
# 크롬 브라우저 닫기     
browser.close()

# url 리스트 출력 
url_list = url_list[:articles_number]
print("크롤링할 글 수: ", len(url_list))
url_list

/var/folders/_v/797cd3nd5377_6_vchd1z9c80000gp/T/ipykernel_4042/1561093043.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  crawling_list = browser.find_elements_by_tag_name('h3 > a')


크롤링할 글 수:  10


['https://www.youtube.com/watch?v=GaRkwOeEr9A',
 'https://www.youtube.com/watch?v=-cEyAQ1EvFc',
 'https://www.youtube.com/watch?v=l4OiMbRXbXQ',
 'https://www.youtube.com/watch?v=v4f_uviLMfA',
 'https://www.youtube.com/watch?v=XTwNzYsXs_g',
 'https://www.youtube.com/watch?v=64HoeokvPW8',
 'https://www.youtube.com/watch?v=6zZ9NDbXYa4',
 'https://www.youtube.com/watch?v=lp2Gybl6kLg',
 'https://www.youtube.com/watch?v=UMYf-2UC0-E',
 'https://www.youtube.com/watch?v=UGVLm_BaVH4']

### step 2. 크롤링

In [7]:
%%time
dict = {}
count = 0

for url in tqdm(url_list):
    # url 열기
    browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(2)

    browser.execute_script("window.scrollTo(0, 400)")
    time.sleep(1)
    
    info = browser.find_element_by_css_selector('.style-scope ytd-video-primary-info-renderer').text.split('조회수')
    info_2 = browser.find_element_by_css_selector('.style-scope ytd-video-owner-renderer').text.split('구독자')
    
    # 제목
    title = info[0].split('\n')[-2]
    
    # 채널명
    chanel = info_2[0].split('\n')[0]
    
    # 조회수, 날짜 위치 추출
    for i in info[1].split('\n'):
        if '회' in i:
            view_date = i

    # 조회수
    #view = int(view_date[0].split('•')[0].replace('회', '').replace(',',''))
    view = int(view_date.split('.')[0].replace('회', '').replace(',',''))
    # 날짜
    #date = view_date[0].split('•')[1]
    date = view_date.split('회')[1]
    # 좋아요
    like = info[1].split('\n')[1]
    # 싫어요
    dont_like = info[1].split('\n')[2]

    try:
        # 댓글 수
        review_cnt = browser.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/ytd-comments/ytd-item-section-renderer/div[1]/ytd-comments-header-renderer/div[1]/h2/yt-formatted-string/span[2]').text.replace(',', '')
        print('<',count,'>', title, '( 댓글 수:', review_cnt,')')
        
        # 댓글 스크롤 다운
        for i in range(math.ceil((int(review_cnt)-20)/20)):
            scroll_down(browser)
        print('1. 스크롤 다운 완료')
        
        # 광고 끄기
        time.sleep(10)
        try:
            browser.find_element_by_css_selector("#main > div > ytd-button-renderer").click()
            print('2. 광고 창 제거함')
        except:
            print('2. 광고 창 안뜸')
        
        try:
            # 답글 n개 보기 클릭
            buttons = browser.find_elements_by_css_selector("#more-replies > a")
            for button in buttons:
                button.send_keys(Keys.ENTER)
                time.sleep(1)
                button.click()
            time.sleep(1)
            print('3. 답글 보기 클릭 완료')
        except:
            print('3. 답글 보기 없음')
        
        try:
            # 답글 더보기 클릭
            buttons_more = browser.find_elements_by_css_selector('#continuation > yt-next-continuation > tp-yt-paper-button')
            for button in buttons_more:
                button.send_keys(Keys.ENTER)
                time.sleep(1)
            print('4. 답글 더보기 클릭 완료')
        except:
            print('4. 답글 더보기 없음')
        
        # 댓글 수집
        review_list = []
        reviews = browser.find_elements_by_css_selector('#content-text')
        for review in reviews:
            review = review.text
            review_list.append(review)
        print('5. 댓글 수집 :', len(review_list))
            
    except:
        review_cnt = ''
        review_list = ''
        print('댓글 차단')
            
    target_info = {}
    target_info['제목'] = title
    target_info['채널명'] = chanel
    target_info['조회수'] = view
    target_info['날짜'] = date
    target_info['좋아요'] = like
    target_info['싫어요'] = dont_like
    target_info['댓글 수'] = review_cnt
    target_info['댓글'] = review_list

    dict[count] = target_info

    browser.close()
    count += 1
    time.sleep(1)

  0%|                                                    | 0/10 [00:00<?, ?it/s]<timed exec>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
<timed exec>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
<timed exec>:40: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


< 0 > 흰색 무지티 5종 추천한다 반팔 긴팔 다 | 가성비, 베이직템, 화이트티, 이너티, 흰티,리뷰 ( 댓글 수: 295 )
1. 스크롤 다운 완료


<timed exec>:51: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
<timed exec>:58: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


2. 광고 창 안뜸
3. 답글 보기 클릭 완료
4. 답글 더보기 클릭 완료


<timed exec>:70: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
<timed exec>:80: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


...

2. 광고 창 안뜸
3. 답글 보기 클릭 완료
4. 답글 더보기 클릭 완료


<timed exec>:70: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
<timed exec>:80: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


5. 댓글 수집 : 111


100%|███████████████████████████████████████████| 10/10 [12:22<00:00, 74.24s/it]

CPU times: user 1.74 s, sys: 305 ms, total: 2.04 s
Wall time: 12min 22s


In [8]:
# 판다스화
df = pd.DataFrame.from_dict(dict, 'index')
df

,제목,채널명,조회수,날짜,좋아요,싫어요,댓글 수,댓글
0,"흰색 무지티 5종 추천한다 반팔 긴팔 다 | 가성비, 베이직템, 화이트티, 이너티,...",오늘의 주우재,4972612021,2021. 2. 26.,9.2천,싫어요,295,[주우재님 남자 엔터테이너 선호도 1위 등극하신거 축하드려요!\n50대 선호도 1위...
1,"최고의 무지티를 찾아서😆 feat.여동생 (로맨틱파이어리츠,에두아르도,쿠어,자바나스...",디렉터 짱구대디,1863752021,2021. 5. 13.,1.7천,싫어요,170,[*세탁 퀄리티 테스트를 위해 세탁 후 건조기에 수축 실험(티셔츠에게는 극한..)을...
2,"체형별, 핏별 흰티 추천 들어간다",핏더사이즈,3908412021,2021. 4. 29.,5.5천,싫어요,343,[핏더형이 작년에도 샤카웨어 추천해줘서 작년에 사입고 진짜 잘 입고 있어!! \n작...
3,만오천원이하 생필품 무지티셔츠 대전- 승자는?,런업,2016172021,2021. 5. 3.,5.3천,싫어요,516,[제가 딱 찾던 리뷰입니다... 역시 런업 형님..\n\n마플에서 매년 반팔 티셔츠...
4,무지티... 직접 사서 입어봤습니다!/ 무지티 4종 비교 분석 by 기방시(레이어드...,기방시 kivanchy,1092372020,2020. 4. 17.,909,싫어요,105,"[이런 비슷한 제품 비교 컨텐츠 너무 도움 되네요 😁😁😁, 여름 필수품 흰색티를 이..."
5,"""다같은 흰티가 아니다!"" ""이건 사지마~""5가지 필수핏(2천원~13만원대) B.P...",Bora Claire,4942722020,2020. 4. 18.,9.3천,싫어요,542,[:와........이 언니는 진짜 찐인듯ㅠㅠ가끔 패션관련 영상 검색할 때 항상 등...
6,2021 남자 무지 반팔티 이거 보면 끝! 그냥 여기서 사!,패션튜브삭형,537072021,2021. 5. 26.,815,싫어요,117,[※유료광고 및 PPL이 포함되어 있지 않습니다.\n\n반팔무지티는 남자분들 여름에...
7,"[TMI리뷰] 여름 필수 인기 무지반팔 4종! 재질,가격 비교분석&추천!",패션회사 영신빌딩,349712020,2020. 4. 27.,208,싫어요,67,[페플TV 담당자입니다!\n영상에 당첨 발표일을 깜빡하고 넣지 않아서\n고정댓글로 ...
8,2020 무지티 완벽정리! 직접 세탁까지 해봤음🔥,스타일가이드 최겨울,747382020,2020. 7. 10.,1.1천,싫어요,73,[트리플에이\n\nhttps://store.musinsa.com/app/produc...
9,2021 무지 반팔티 이 영상으로 끝낸다. | 여름 무지 반팔티 총정리 리뷰!!,진진호,907692021,2021. 5. 11.,1.1천,싫어요,111,[• 섭외 브랜드들의 제작지원으로 제작된 유료광고 영상입니다\n\n\n\n\n\n1...


In [9]:
# 데이터 프레임 -> 엑셀 파일로 저장
df.to_excel('crawler_youtube_review_{}.xlsx'.format(keyword))

### 유튜브 디테일 자료 크롤링 ver. 3 과정 및 결과 

#### 1. 과정 
1. 검색어와 크롤링 할 갯수 지정 
2. 크롤링 후 URL 저장 
3. 저장 된 URL (해당 채널)로 들어가서 아래의 내용 크롤링 
    - 콘텐츠 타이틀
    - 채널명
    - 조회수
    - 날짜
    - 좋아요 수
    - 싫어요 수
    - 댓글 수
    - 댓글 
4. 데이터 프레임으로 저장 
5. 데이터 프레임 -> 엑셀로 저장 

#### 2. 결과 
- 콘텐츠 타이틀 불러오기 성공 
- 조회수 불러오기 성공
- 날짜 불러오기 성공 
    - 초반에 날짜가 월 기준만 나온 문제 해결 (현재 날짜 모두 나옴)
- 좋아요 / 싫어요 수 불러오기 성공 
- 댓글수 불러오기 성공
- 댓글 불러오기 성공 

#### 3. 아쉬운 및 개선점
1. 아쉬운 점은 좋아요 수가 int가 아니라 str 형식으로 되어 있음 ( 92,000[X] / 9.2천[O])  -> 논의 필요
2. 댓글 크롤링 완료 하지만 댓글 모두를 크롤링하기에는 컴퓨터 자원이 부족하여 데이터 모두 받지 못함. -> 추후 어떻게 할지 논의 필요
    - 해당 문제 뿐만 아니라 컴퓨팅 자원 및 IP 주소가 막힐 까봐 많은 양의 데이터를 크롤링 하지 못할 것 같은 걱정이 있음. 
3. 각각의 댓글을 이어 붙여서 가독성이 좋지 않음. -> 논의 필요
4. 더 필요한 정보가 있으면 추가로 크롤링하기